# Local Inference with trained ALBERT model

Let's first import all the required packages.

In [ ]:
from metaflow import Flow, get_metadata
from determined.experimental import Determined
import torch

We can interact with Metaflow directly and return the most recent run of the DeterminedFlow:

In [ ]:
run = Flow('DeterminedFlow').latest_successful_run
print(f"Using run: {str(run)}")

## Populate Determined fields with Metaflow run

Since our DeterminedFlow tracks the experiment run on Determined, we can easily retrieve the experiment id and the cluster url that the experiment was run on.

In [ ]:
master_url = run.data.det_master
experiment_id = run.data.experiment_id

print(f"master_url: {master_url}")
print(f"experiment_id: {experiment_id}")

## Load best performing checkpoint from Determined

Using this information, we can retrieve the top checkpoint for the experiment...

In [ ]:
checkpoint = Determined(master=master_url). \
    get_experiment(experiment_id). \
    top_checkpoint()

...and load it as a model for running inference.

In [ ]:
model = checkpoint.load(map_location=torch.device('cpu'))

## Use model for prediction locally

We'll first import our predict function.

In [ ]:
from predict import run_prediction

Then set the context and question...

In [ ]:
context = "New Zealand is a sovereign island country in the southwestern Pacific Ocean. It has a total land area of 268,000 square kilometres and a population of 4.9 million."
questions = ["How many people live in New Zealand?"]

...and finally run the prediction:

In [ ]:
predictions = run_prediction(model, questions, context)

In [ ]:
for key in predictions.keys():
    print(predictions[key])

Feel free to replace the `context` and `question` above and re-run to see how your model performs on different sets of questions!

### Conclusion

As you can see in this short notebook, training your model with Determined and Metaflow gives you the advanced capabilities of the Determined platform while maintaining a flexible workflow with Metaflow.